In [1]:
import cv2
import math
import numpy as np
import uuid
import csv
from tqdm import tqdm

np.set_printoptions(precision=8)

In [2]:
def draw_first_scale(img, center, radius, arc_angles, color):
    # Unpack center coordinates
    xc, yc = center
    # Unpack angles
    start_angle, end_angle = arc_angles

    # cv2.circle(img, (xc, yc), 10, (255, 0, 0), cv2.FILLED, cv2.LINE_AA)

    # Draw scale line
    # # cv2.circle(gauge_background, (xc, yc), r_scale1, scale1_color, 1, cv2.LINE_AA)
    cv2.ellipse(img, (xc, yc), (radius, radius), 0, start_angle, end_angle, color, 1, cv2.LINE_AA)

    # Draw scale's marks
    main_mark_len = 15
    sub_mark_len = 9
    main_marks_qty = 10
    sub_marks_qty = 10
    total_marks_qty = main_marks_qty * sub_marks_qty
    mark_delta_angle = abs(end_angle-start_angle)/total_marks_qty

    # Font Parameters
    font_face = cv2.FONT_HERSHEY_PLAIN
    font_scale = 1.0
    font_thickness = 2

    first_point = (0, 0)
    last_point = (0, 0)

    draw_main = True
    sub_count = 0
    for i in range(0, total_marks_qty+1):
        if draw_main:
            # Calculate coordinates
            start_point = (xc + int(radius * math.cos(math.radians(start_angle + i * mark_delta_angle))),
                           yc + int(radius * math.sin(math.radians(start_angle + i * mark_delta_angle))))
            end_point = (xc + int((radius + main_mark_len) * math.cos(math.radians(start_angle + i * mark_delta_angle))),
                         yc + int((radius + main_mark_len) * math.sin(math.radians(start_angle + i * mark_delta_angle))))
            cv2.line(img, start_point, end_point, color, 3, cv2.LINE_AA)

            if i == 0:
                first_point = start_point
            if i == total_marks_qty:
                last_point = start_point

            # Put Text
            # 1. Get text dim
            text_size, _ = cv2.getTextSize(f"{i}", font_face, font_scale, font_thickness)
            tw, th = text_size
            # 2. Calculate bottom-left coordinate
            tx = xc + int((radius + 30) * math.cos(math.radians(start_angle + i * mark_delta_angle))) - int(tw/2)
            ty = yc + int((radius + 30) * math.sin(math.radians(start_angle + i * mark_delta_angle))) + int(th/2)
            start_point = (tx, ty)
            # 3. Draw text
            cv2.putText(img, f"{i}", start_point, font_face, font_scale, color, font_thickness, cv2.LINE_AA)

            draw_main = False
            sub_count = 0
        else:
            delta_radius = 0
            if sub_count == (sub_marks_qty // 2)-1:
                delta_radius = 5
            # Calculate coordinates
            start_point = (xc + int(radius * math.cos(math.radians(start_angle + i * mark_delta_angle))),
                           yc + int(radius * math.sin(math.radians(start_angle + i * mark_delta_angle))))
            end_point = (xc + int((radius + sub_mark_len + delta_radius) * math.cos(math.radians(start_angle + i * mark_delta_angle))),
                         yc + int((radius + sub_mark_len + delta_radius) * math.sin(math.radians(start_angle + i * mark_delta_angle))))
            cv2.line(img, start_point, end_point, color, 1, cv2.LINE_AA)
            sub_count += 1
            if sub_count == sub_marks_qty-1:
                draw_main = True

    return center, first_point, last_point

In [3]:
def draw_second_scale(img, center, radius, arc_angles, color):
    # Unpack center coordinates
    xc, yc = center
    # Unpack angles
    start_angle, end_angle = arc_angles

    # cv2.circle(img, (xc, yc), 10, (255, 0, 0), cv2.FILLED, cv2.LINE_AA)

    # Draw scale line
    # # cv2.circle(gauge_background, (xc, yc), r_scale1, scale1_color, 1, cv2.LINE_AA)
    cv2.ellipse(img, (xc, yc), (radius, radius), 0, start_angle, end_angle, color, 1, cv2.LINE_AA)

    # Draw scale's marks
    main_mark_len = 15
    sub_mark_len = 9
    main_marks_qty = 7
    sub_marks_qty = 10
    total_marks_qty = main_marks_qty * sub_marks_qty
    mark_delta_angle = abs(end_angle-start_angle)/total_marks_qty

    # Font Parameters
    font_face = cv2.FONT_HERSHEY_PLAIN
    font_scale = 1.0
    font_thickness = 2

    draw_main = True
    sub_count = 0
    for i in range(0, total_marks_qty+1):
        if draw_main:
            # Calculate coordinates
            start_point = (xc + int(radius * math.cos(math.radians(start_angle + i * mark_delta_angle))),
                           yc + int(radius * math.sin(math.radians(start_angle + i * mark_delta_angle))))
            end_point = (xc + int((radius - main_mark_len) * math.cos(math.radians(start_angle + i * mark_delta_angle))),
                         yc + int((radius - main_mark_len) * math.sin(math.radians(start_angle + i * mark_delta_angle))))
            cv2.line(img, start_point, end_point, color, 3, cv2.LINE_AA)

            # Put Text
            # 1. Get text dim
            text_size, _ = cv2.getTextSize(f"{i//10}", font_face, font_scale, font_thickness)
            tw, th = text_size
            # 2. Calculate bottom-left coordinate
            tx = xc + int((radius - 30) * math.cos(math.radians(start_angle + i * mark_delta_angle))) - int(tw/2)
            ty = yc + int((radius - 30) * math.sin(math.radians(start_angle + i * mark_delta_angle))) + int(th/2)
            start_point = (tx, ty)
            # 3. Draw text
            cv2.putText(img, f"{i//10}", start_point, font_face, font_scale, color, font_thickness, cv2.LINE_AA)

            draw_main = False
            sub_count = 0
        else:
            delta_radius = 0
            if sub_count == (sub_marks_qty // 2)-1:
                delta_radius = 5
            # Calculate coordinates
            start_point = (xc + int(radius * math.cos(math.radians(start_angle + i * mark_delta_angle))),
                           yc + int(radius * math.sin(math.radians(start_angle + i * mark_delta_angle))))
            end_point = (xc + int((radius - sub_mark_len - delta_radius) * math.cos(math.radians(start_angle + i * mark_delta_angle))),
                         yc + int((radius - sub_mark_len - delta_radius) * math.sin(math.radians(start_angle + i * mark_delta_angle))))
            cv2.line(img, start_point, end_point, color, 1, cv2.LINE_AA)
            sub_count += 1
            if sub_count == sub_marks_qty-1:
                draw_main = True

In [4]:
def draw_gauge_with_scale1(img, center, radius, arc_angles, color, value):
    # Unpack center coordinates
    xc, yc = center
    # Unpack angles
    start_angle, end_angle = arc_angles
    # Scale Bounds
    start_value = 0.0
    end_value = 100.0

    # Calculate angle for the value
    value_angle = value * ((end_angle-start_angle)/(end_value-start_value))
    # Adjust to start in start_angle
    value_angle += start_angle
    #
    xv = xc + int(radius * math.cos(math.radians(value_angle)))
    yv = yc + int(radius * math.sin(math.radians(value_angle)))
    #
    cv2.line(img, (xc, yc), (xv, yv), color, 5, cv2.LINE_AA)
    cv2.circle(img, (xc, yc), 10, color, cv2.FILLED)

    # # Calculate line "y = m1*x + b1" through (xc, yc) and (xv, yv)
    # m1 = (yc - yv) / (xc - xv)
    # b1 = (xc*yv-yc*xv) / (xc - xv)
    # # Calculate perpendicular line through (xc, yc)
    # m2 = -1/m1
    # b2 = yc - m2*xc
    #
    # # # testing
    # # xv = int(xc + radius//2)
    # # yv = int(m1*xv+b1)
    # # cv2.circle(img, (xv, yv), 10, (0, 255, 255), cv2.FILLED)
    # #
    # # xv = int(xc + radius//2)
    # # yv = int(m2*xv+b2)
    # # cv2.line(img, (xc, yc), (xv, yv), (0, 255, 255), 3, cv2.LINE_AA)

    # print(f"y = {m1:0.3f}*x + {b1:0.3f}")
    # print(f"y = {m2:0.3f}*x + {b2:0.3f}")
    return (xv, yv)


# Main

In [5]:
# Load background image
gauge_background = cv2.imread('images/pressure-gauge.png')
# Get Shape of gauge_background (gb)
gb_h, gb_w, gb_c = gauge_background.shape
print(f"Background Image Size: {gauge_background.shape}")

# Coordinates of the center
xc, yc = 253, 202

# # Radius of the center point
# rc_min = 10
# Greater Radius
# rc_max = 150
rc_max = 160
# First Scale Radius
r_scale1 = 113
# Second Scale Radius
r_scale2 = 107

# Background color for the generated gauge
background_color = (209, 207, 206)
# background_color = (239, 237, 236)
# First Scale color
scale1_color = (0, 0, 124)
# Second Scale color
scale2_color = (0, 0, 0)
# Gauge color
gauge_color = (100, 0, 200)

for value in tqdm(range(0, 1000), desc="Generating Dataset", ncols=80):
    # Load background image
    gauge_background = cv2.imread('images/pressure-gauge.png')
    # Draw the circle to prepare the "canvas"
    cv2.circle(gauge_background, (xc, yc), rc_max, background_color, cv2.FILLED, cv2.LINE_AA)

    # First Scale Drawing
    center, mark1, mark2 = draw_first_scale(gauge_background, (xc, yc), r_scale1, (137, 407), scale1_color)
    # Second Scale Drawing
    draw_second_scale(gauge_background, (xc, yc), r_scale2, (137, 407), scale2_color)
    # Draw the gauge
    # scale1_value = np.random.uniform(low=0.0, high=100.0)
    scale1_value = value * 0.1
    mark_value = draw_gauge_with_scale1(gauge_background, (xc, yc), r_scale1+10, (137, 407), gauge_color, scale1_value)

    # Generate image file and annotation
    filename = uuid.uuid1()
    cv2.imwrite(f'dataset/{filename}.jpg', gauge_background)
    annotation = np.array([center[0], center[1], mark1[0], mark1[1], mark2[0], mark2[1],
                           mark_value[0], mark_value[1]], dtype='float64') / 500.0
    with open(f'dataset/{filename}.txt', 'w') as csvfile:
        writer = csv.writer(csvfile, dialect='excel', delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        writer.writerow([f'{annotation[0]:0.8f}', f'{annotation[1]:0.8f}', f'{annotation[2]:0.8f}',
                         f'{annotation[3]:0.8f}', f'{annotation[4]:0.8f}', f'{annotation[5]:0.8f}',
                         f'{annotation[6]:0.8f}', f'{annotation[7]:0.8f}'])

    cv2.imshow("Preview", gauge_background)
    key = cv2.waitKey(1) & 0xFF
    if key == 27 or key == ord('q'):
        break

# Gracefully shutdown
cv2.destroyAllWindows()

Background Image Size: (500, 500, 3)


Generating Dataset: 100%|██████████████████| 1000/1000 [00:09<00:00, 108.84it/s]
